Import the Hyperspy Function Library

In [1]:
import matplotlib
matplotlib.rcParams["backend"] = 'Qt4Agg'
import hyperspy.api as hs
import numpy as np
%matplotlib qt4

# Step 1: Import and Calibrate the Experimental Data

Opens a load window. Use hs.load('filename') if filename is known specifically.

In [17]:
s = hs.load()
s

[<Signal2D, title: HAADF, dimensions: (|512, 512)>,
 <EDSSEMSpectrum, title: EDX, dimensions: (512, 512|2048)>]

These next two lines are needed for .bcf files from Bruker/Espirit as the imported data comes as a list containing both the ADF image and EDX spectral data cube.

In [18]:
image = s[0]
s = s[1]
s

<EDSSEMSpectrum, title: EDX, dimensions: (512, 512|2048)>

In [21]:
s.set_signal_type('EDS_TEM')
s.metadata.General.title = 'Original Data Cube'
s.metadata

├── Acquisition_instrument
│   └── TEM
│       ├── Detector
│       │   └── EDS
│       │       ├── azimuth_angle = 45.0
│       │       ├── detector_type = SuperX
│       │       ├── elevation_angle = 18.0
│       │       ├── energy_resolution_MnKa = 130.0
│       │       └── live_time = 1
│       ├── beam_energy = 200
│       ├── stage_x = None
│       ├── stage_y = None
│       └── tilt_stage = 0.0
├── General
│   ├── datetime = datetime.datetime(2016, 6, 14, 12, 28, 54)
│   ├── original_filename = 04.bcf
│   └── title = Original Data Cube
├── Sample
│   ├── elements = ['Ni', 'Pt']
│   ├── name = Image 1942
│   └── xray_lines = ['Ni_Ka', 'Pt_La']
└── Signal
    ├── binned = True
    └── signal_type = EDS_TEM

Adjust the metadata for all the microscope parameters.

In [68]:
s.set_microscope_parameters(beam_energy = 200)
s.set_microscope_parameters(live_time = 1)
s.set_microscope_parameters(real_time = 0.0136535) #dwell time per pixel
s.set_microscope_parameters(beam_current = 0.068) #in nA?
s.isig[50:]

#Or open a GUI using s.set_microscope_parameters(). or hs.preferences.gui()

<EDSTEMSpectrum, title: Original Data Cube, dimensions: (512, 512|1998)>

In [67]:
s.axes_manager[0].name = 'x'
s.axes_manager[1].name = 'y'
s.axes_manager

<Axes manager, axes: (512, 512|2048)>
            Name |   size |  index |  offset |   scale |  units 
================ | ====== | ====== | ======= | ======= | ====== 
               x |    512 |      0 |       0 | 9.7e-05 |     µm 
               y |    512 |      0 |       0 | 9.7e-05 |     µm 
---------------- | ------ | ------ | ------- | ------- | ------ 
          Energy |   2048 |        |   -0.47 |    0.01 |    keV

Opens three windows to edit the scale and offset of each axis including the EDX Energy axis.

In [69]:
s.axes_manager.gui()
image.axes_manager.gui()
#s.get_calibration_from(s1pixel) will take all the calibration 

Plotting both the image and the spectrum to see that they are imported correctly.

In [23]:
s.plot()
image.plot()

Setting the elements to be used for quantification. Make sure that these match those used for calibration samples.

In [70]:
s.set_elements(['Pt','Ni'])
s.set_lines(['Pt_La', 'Ni_Ka'])
s.metadata.Sample

├── elements = ['Ni', 'Pt']
├── name = Image 1942
└── xray_lines = ['Ni_Ka', 'Pt_La']

# Step 2: Import Calibration Dictionary

Import the data.

In [27]:
cal = hs.load()
calibration = cal.metadata.calibration

Check which elements are included within the calbration file.

In [28]:
calibration.keys()

['Ni', 'Pt']

In [59]:
cal_Ni = calibration['Ni']
cal_Pt = calibration['Pt']
cal_Ni

├── Background windows = array([[ 6.61175942,  6.75614952,  7.76688019,  7.91127029]])
├── Integration windows = [[7.333709903386694, 7.622490096613307]]
├── cross_section = 8.4899126303055183
└── lines = CoKa

In [35]:
iw = cal_Ni['Integration windows'] + cal_Pt['Integration windows']
iw

[[7.333709903386694, 7.622490096613307], [9.281605451805987, 9.65]]

In [54]:
bw = np.append(cal_Ni['Background windows'], 
               cal_Pt['Background windows'],
              axis=0)
bw[0][2] = bw[1][0]
bw[0][3] = bw[1][1]
bw

array([[ 6.61175942,  6.75614952,  8.47913271,  8.63962726],
       [ 8.47913271,  8.63962726,  9.7630891 ,  9.92358364]])

In [60]:
cs = cal_Ni['cross_section'],cal_Pt['cross_section']
cs

(8.4899126303055183, 7.8147609092183261)

Check that the backgroun windows and integration windows still work on the experimental data. As the calibration samples are single element, they may have selected windows which overlap with other elemental peaks which are in the final experimental data. 

If you do need to change bw and iw here, it is advised to repeat the wedge calibration workbook so that both sets of integration windows match completely. Background windows are less critical but integration windows should be kept the same between calibration and experiment.

In [71]:
s.sum().plot(True, background_windows=bw, integration_windows=iw)

//anaconda/lib/python3.5/site-packages/matplotlib/__init__.py:892: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


# Step 3: Quantify the Experimental Data, using s.quantification()

If the spectrum requires binnning, it is better to do this now before extracting the intensities.

In [78]:
#Need to insert non-integer binning here.

In [72]:
inten = s.get_lines_intensity(background_windows=bw, integration_windows=iw)

In [73]:
p = s.quantification(inten, 'cross_section', cs)

Subsequent processing and image plotting and of the resulting data.

In [74]:
Total = p[1][0] + p[1][1]

In [75]:
Total.plot()

In [80]:
Total[1][1].plot()

TypeError: 'Signal2D' object does not support indexing

In [79]:
for i in Total:
    for y in i:
        if y <= 0.4:
            Total[i][y] = 0

Total.plot()

TypeError: 'Signal2D' object is not subscriptable